Get MM1 scripts and error bars running properly for both:
  (1) non-biomarker sex-div
  (2) biomarker pre- vs. post-meno

In [12]:
require('R.utils')
require('data.table')


source("../mixture_model_scripts/model_utils.R")
source("../mixture_model_scripts/heritability_utils.R")

DATA.FOLDER <- "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/1009/"


In [1]:
model <- 1
biomarker <- FALSE
calcLogLik <- FALSE
trait.type <- 'quant'
ndim <- 2
downsampled <- FALSE

maf.cutoff <- 0.01
se.cutoff <- 0.2

trait <- "whr"
load("../data/biomarker/dat_HDL_cholesterol.RData")
load("../data/biomarker/f_HDL_cholesterol.RData")

In [6]:
m1.pi <- getPi(fit1)
    m1.Sigma <- getSigmaMulti(fit1, ndim)
    rg <- getRgMulti(fit1, ndim)
    rg.c <- getRgConfMulti(fit1, ndim)

    # assign each SNP to a component, estimate heritability
    dat <- labelCategories(dat, m1.Sigma, m1.pi) # label the SNPs with heritability
    h <- overallHeritability(dat, m1.Sigma, m1.pi)
   

    # write out the data
    next.row <- data.frame(t(c(trait, dat$dat$N, unlist(m1.pi), unlist(m1.Sigma), unlist(rg), unlist(rg.c$l), unlist(rg.c$u), unlist(h))))

In [7]:
next.row

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13
HDL_cholesterol,328861,0.951617341938868,0.0483826580611323,0.000358464919662782,0.000348544253009364,0.000348544253009364,0.000362175468303302,0.967351097636642,0.960959475566194,0.973712391093405,0.134767295936721,0.138863048988029


In [10]:
# extract all estimate
        list_of_draws <- rstan::extract(fit1)
        pi.draws <- list_of_draws$pi
        p <- pi.draws
        s.draws <- list_of_draws$Sigma
        Sigma <- s.draws

        # extract lower + upper pi
        ordered.p <- p[order(p[,2]),] # ordering p by the non-null component 
        p.lower <- ordered.p[0.025*nrow(ordered.p),]
        p.upper <- ordered.p[0.975*nrow(ordered.p),]

        # extract lower + upper sigma
        ordered.S <- Sigma[order(Sigma[,1,1]),,] # double check - is this the right sigma to extract?? I think so
        s.upper <- ordered.S[0.975*dim(Sigma)[1],,] 
        s.lower <- ordered.S[0.025*dim(Sigma)[1],,]



In [11]:
dat2 <- dat
        dat2$categories <- NULL
        dat.u <- labelCategories(dat2, s.upper, p.upper)
        dat.l <- labelCategories(dat2, s.lower, p.lower)

        h.up <- overallHeritability(dat.u, s.upper, p.upper)
        h.low <- overallHeritability(dat.l, s.lower, p.lower)
        res <- list("up"=h.up, "low"=h.low)

        # reformat into data frame

     my.df <- cbind(t(as.data.frame(res)), trait)
        my.df2 <- data.frame(cbind(my.df, rownames(my.df)))

    colnames(my.df2) <- c("hf", "hm", "trait", "int")
    my.df3 <- melt(my.df2, id.vars=c("trait", "int"), variable.name="sex")
         rownames(my.df3) <- NULL


Warning message:
“attributes are not identical across measure variables; they will be dropped”

In [15]:
head(my.df3 )
dim(Sigma)

trait,int,sex,value
HDL_cholesterol,up,hf,0.147009997469867
HDL_cholesterol,low,hf,0.125813042811673
HDL_cholesterol,up,hm,0.152299297968592
HDL_cholesterol,low,hm,0.128457696922574


[1] 1600    2    2

In [16]:
chrs <- c(1:22, "X", "XY") # make sure works for X, XY

ndim_to_prefixes <- list("2"=c("zerosex", "onesex"), 
    "3"=c("pre_meno", "post_meno", "onesex"),
    "4"=c("under65_f", "over65_f", "under65_m", "over65_m"))

list.prefixes <- ndim_to_prefixes[[as.character(ndim)]]

if (downsampled==TRUE){ 
    list.prefixes <- sapply(list.prefixes, function(prefix) paste(prefix, "d", sep="_"))
}

downsampled_str <- ifelse(downsampled, "downsampled", "")
print(sprintf("Running M1 for trait %s with %s dim, %s, with prefixes %s", trait, ndim, downsampled_str, paste(list.prefixes, collapse=" ")))


[1] "Running M1 for trait HDL_cholesterol with 2 dim, , with prefixes zerosex onesex"


appears that things work!

In [23]:
loadDat <- function(trait, trait.type){
    
    if (biomarker==TRUE){
     list.ds <- lapply(c("female", "male"), function(sex) loadBiomarkerDat(trait, sex))
    } else {
    # for each trait
    list.ds <- lapply(list.prefixes, function(prefix) {
        all.dat <- do.call(rbind, lapply(21:22, function(chr) { getFile(prefix, chr, trait)}));
        colnames(all.dat)[1:3] <- c("CHR", "BP", "SNP");
        return(all.dat)
    })
    }

    list.ds2 <- extractOverlappingRows(list.ds)

    stan.obj <- extractDataStanMulti(list.ds2)
    return(stan.obj)
}
loadDat("whr", "quant")

[1] "File missing for c21 trait:whr zerosex"
[1] "File missing for c22 trait:whr zerosex"
[1] "File missing for c21 trait:whr onesex"
[1] "File missing for c22 trait:whr onesex"
[1] 2
[1] 6169   11


0.043321900,-1.77673e-01
-0.014716900,4.06143e-02
-0.004846330,-8.40607e-03
0.004277180,-1.75870e-04
-0.000639981,-5.53760e-04
-0.000647245,-2.32051e-05
0.013883200,1.65215e-02
-0.003132010,-1.13569e-02
-0.002816850,-2.37469e-03
-0.006700660,2.36435e-04
-0.002867860,1.58557e-03


In [18]:
getFile

function (dat.source, chr, field) 
{
    prefix <- sprintf("%sukb24893_v2.%s", GWAS.FOLDER, field)
    file.dat <- paste(c(prefix, ".", dat.source, ".", field, 
        "_c", chr, ".glm.linear.gz"), collapse = "")
    my.classes = c("character", "numeric", "character", "character", 
        "character", "character", "numeric", "numeric", "numeric", 
        "numeric", "numeric")
    col.labels <- c("CHROM", "POS", "ID", "REF", "ALT1", "TEST", 
        "OBS_CT", "BETA", "SE", "T_STAT", "P")
    checks <- fileChecks(file.dat, dat.source, chr, field)
    if (checks == -1) {
        file.dat <- paste(c(prefix, ".", dat.source, ".", "PHENO1", 
            "_c", chr, ".glm.linear.gz"), collapse = "")
        checks <- fileChecks(file.dat, dat.source, chr, field)
        if (checks == -1) {
            return(NA)
        }
    }
    dat <- read.table(file.dat, colClasses = my.classes, header = FALSE)
    colnames(dat) <- col.labels
    dat.1 <- dat[dat$TEST == "ADD", ]
    rownames(dat.1) <- dat.1$ID
    dat.2 <- dat.1[!is.na(dat.1$SE), ]
    dat.3 <- dat.2[dat.2$SE < QUANT.SE.CUTOFF, ]
    dat.4 <- dat.3[dat.3$ID %in% snps.to.keep$V1, ]
    return(dat.4)
}
<bytecode: 0x1080ac48>

In [22]:
prefix <- sprintf("%sukb24893_v2.%s", GWAS.FOLDER, "whr")
print(prefix)

[1] "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/gwas1009/ukb24893_v2.whr"


In [3]:

model <- 1
biomarker <- FALSE
calcLogLik <- FALSE
trait.type <- 'quant'
ndim <- 3
downsampled <- FALSE

maf.cutoff <- 0.01
se.cutoff <- 0.2

trait <- "Testosterone"



In [6]:
biomarker <- FALSE
maf.cutoff <- 0.01
se.cutoff <- 0.2

chrs <- c(1:22, "X", "XY") # make sure works for X, XY

ndim_to_prefixes <- list("2"=c("zerosex", "onesex"), 
    "3"=c("pre_meno", "post_meno", "onesex"),
    "4"=c("under65_f", "over65_f", "under65_m", "over65_m"))

list.prefixes <- ndim_to_prefixes[[as.character(ndim)]]

if (downsampled==TRUE){ 
    list.prefixes <- sapply(list.prefixes, function(prefix) paste(prefix, "d", sep="_"))
}

downsampled_str <- ifelse(downsampled, "downsampled", "")
print(sprintf("Running M1 for trait %s with %s dim, %s, with prefixes %s", trait, ndim, downsampled_str, paste(list.prefixes, collapse=" ")))
print(list.prefixes)

[1] "Running M1 for trait Testosterone with 3 dim, , with prefixes pre_meno post_meno onesex"
[1] "pre_meno"  "post_meno" "onesex"   


In [13]:


loadDat <- function(trait, trait.type, gwas.folder){
    
    if (biomarker==TRUE){
     list.ds <- lapply(c("female", "male"), function(sex) loadBiomarkerDat(trait, sex))
    } else {
    # for each trait
    list.ds <- lapply(list.prefixes, function(prefix) {
        all.dat <- do.call(rbind, lapply(chrs, function(chr) { getFile(prefix, chr, trait, gwas.folder)}));
        colnames(all.dat)[1:3] <- c("CHR", "BP", "SNP");
        return(all.dat)
    })
    }

    list.ds2 <- extractOverlappingRows(list.ds)

    stan.obj <- extractDataStanMulti(list.ds2)
    return(stan.obj)
}

dat <- loadDat("Testosterone", trait.type, "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/gwas_age_sex/")
                       
ukb24893_v2_c9.Testosterone.pre_meno.Testosterone.glm.linear


[1] "File missing for c1 trait:Testosterone pre_meno"
[1] "File missing for c1 trait:Testosterone pre_meno"
[1] "File missing for c2 trait:Testosterone pre_meno"
[1] "File missing for c2 trait:Testosterone pre_meno"
[1] "File missing for c3 trait:Testosterone pre_meno"
[1] "File missing for c3 trait:Testosterone pre_meno"
[1] "File missing for c4 trait:Testosterone pre_meno"
[1] "File missing for c4 trait:Testosterone pre_meno"
[1] "File missing for c5 trait:Testosterone pre_meno"
[1] "File missing for c5 trait:Testosterone pre_meno"
[1] "File missing for c6 trait:Testosterone pre_meno"
[1] "File missing for c6 trait:Testosterone pre_meno"
[1] "File missing for c7 trait:Testosterone pre_meno"
[1] "File missing for c7 trait:Testosterone pre_meno"
[1] "File missing for c8 trait:Testosterone pre_meno"
[1] "File missing for c8 trait:Testosterone pre_meno"
[1] "File missing for c9 trait:Testosterone pre_meno"
[1] "File missing for c9 trait:Testosterone pre_meno"
[1] "File missing for c10 tr

ERROR: Error in dimnames(x) <- dn: length of 'dimnames' [2] not equal to array extent
